# Generating the FD inputs for replication of the lab experiment 

**MODEL**: Homogeneous model of size 50x50x50 cm with properties of _Fine Sandstone_ as defined by Xi et al., 2009
- v_p 5.26 km/s 
- v_s 3.26 km/s
- rho 2.59 g/cm^3

Add a buffer of 25 gridpoints around to imitate 

**GEOMETRY**: Imitating 5 DAS loops per face, with loops starting 2cm into face. Loops are imitated by 10 point receivers per face (i.e. every 5cm)

In [11]:
import numpy as np
import os

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [12]:
outpath = '../../../SOFI-modelling/SOFI_LabImitiation/inputs/'

# MODEL

In [13]:
def make_homo_model(dimlims, 
                    deltadims, 
                    subsurface_properties,
                    pad=True, 
                    npad=30,
                    freesurface=True
                   ):
    
    # Assign variables
    xmax = ymax = zmax = dimlims  # m
    dx = dy = dz = deltadims  # m
    
    # Compute number of grid points
    nx = ny = nz = int(xmax/dx) # Grid points
    nx_m = ny_m = nz_m = nx
    if pad:
        nx_m = ny_m = nz_m = int(nx + npad*2) # Grid points
    if freesurface:
        ny_m = int(nx + npad)
        
    # Make models
    vp = subsurface_properties[0]  #m/s
    vs = subsurface_properties[1]  #m/s 
    rho = subsurface_properties[2]; #kg/m3
    
    mod_vp = vp * np.ones([nx_m,ny_m, nz_m])
    mod_vs = vs * np.ones([nx_m,ny_m,nz_m])
    mod_rho = rho * np.ones([nx_m,ny_m,nz_m])
    
    return mod_vp, mod_vs, mod_rho

In [14]:
# Model params
mod_length = 0.5  # m
dx = 0.01  # m
nx = int(mod_length/dx)
npad = 0

# Subsurface properties
vp = 5260 #m/s
vs = 3260 
rho = 2590; #kg/m3

mod_vp, mod_vs, mod_rho = make_homo_model(dimlims=mod_length, 
                                            deltadims=dx, 
                                            subsurface_properties=[vp,vs,rho],
                                            pad=False, 
                                            npad=npad,
                                            freesurface=False
                                           )

print(mod_vp.shape)

mod_path = os.path.join(outpath,'model')
mod_vp.astype(np.float32).tofile(os.path.join(mod_path,'lab_sample.vp'))#,format='<f')
mod_vs.astype(np.float32).tofile(os.path.join(mod_path,'lab_sample.vs'))  #,format='<f')
mod_rho.astype(np.float32).tofile(os.path.join(mod_path,'lab_sample.rho'))  #,format='<f')

(50, 50, 50)


# SOURCE LOCATION

In [15]:
def comp_source_loc(orig_xyz, 
                    deltadims, 
                    npad=30,
                    freesurface=True):
    
    '''convert orig xyz to new xyz taking into account padding'''
    
    # convert to gridpoint
    o_source_gp = [x/dx for x in orig_xyz]
    n_source_gp = [x+npad for x in o_source_gp]
    
    # Account for freesurface
    if freesurface:
        n_source_gp[1] = o_source_gp[1]
        
    # Convert back to meters
    n_source_xyz = [x*dx for x in n_source_gp]
    
    return n_source_xyz

In [16]:
freesurface = True
orig_xyz = [0.25, 0.25, 0.25]

n_source_xyz = comp_source_loc(orig_xyz, 
                    deltadims=dx, 
                    npad=npad,
                    freesurface=False)

print('SOURCE LOC: ',n_source_xyz)

SOURCE LOC:  [0.25, 0.25, 0.25]


# RECEIVERS 

In [17]:
def get_recs_on_face(nrpF, nx, dx, rec_buffer=6, npad=30, freesurface=True):
    '''At the moment identical in x and y '''
    
    # Get grid location without considering padding
    start_point = rec_buffer*dx
    end_point = (nx-(2*rec_buffer))*dx 
    grid_locs = np.linspace(start_point, end_point, nrpF)
    
    if freesurface:  ypad=0    
    else: ypad=npad
    
    if npad==0:
        xstrt0 = ystrt0 = zstrt0 = dx
        xstrt1 = ystrt1 = zstrt1 = (nx-1)*dx
    else:
        xstrt0 = zstrt0 = npad*dx
        xstrt1 = zstrt1 = (npad+nx)*dx
        ystrt0 = ypad*dx
        ystrt1 = (ypad+nx)*dx
        
    # X FACE
    F0_rx, F0_ry, F0_rz = np.meshgrid(xstrt0, grid_locs+(ypad*dx), grid_locs+(npad*dx))      # X face 0 
    F1_rx, F1_ry, F1_rz = np.meshgrid(xstrt1, grid_locs+(ypad*dx), grid_locs+(npad*dx))      # X face 1  
    # Y FACE
    F2_rx, F2_ry, F2_rz = np.meshgrid(grid_locs+(npad*dx), ystrt0, grid_locs+(npad*dx))      # Y face 0 
    F3_rx, F3_ry, F3_rz = np.meshgrid(grid_locs+(npad*dx), ystrt1, grid_locs+(npad*dx))      # Y face 1 
    # Z FACE
    F4_rx, F4_ry, F4_rz = np.meshgrid(grid_locs+(npad*dx), grid_locs+(ypad*dx), zstrt0)   # Z face 0 
    F5_rx, F5_ry, F5_rz = np.meshgrid(grid_locs+(npad*dx), grid_locs+(ypad*dx), zstrt1)  # Z face 1 

    # COMBINE
    grid_rx = np.vstack([rx.flatten() for rx in [F0_rx,F1_rx,F2_rx,F3_rx,F4_rx,F5_rx,]])
    grid_ry = np.vstack([ry.flatten() for ry in [F0_ry,F1_ry,F2_ry,F3_ry,F4_ry,F5_ry,]])
    grid_rz = np.vstack([rz.flatten() for rz in [F0_rz,F1_rz,F2_rz,F3_rz,F4_rz,F5_rz,]])

    gridded_recs = np.vstack((grid_rx.flatten(), grid_ry.flatten(), grid_rz.flatten()))
    nr = gridded_recs.shape[1]
    
    return gridded_recs,nr

In [18]:
gridded_recs,nr = get_recs_on_face(nrpF=5, nx=nx, dx=dx, rec_buffer=6, npad=npad, freesurface=False)

In [19]:
print (nr, gridded_recs.shape)
print(gridded_recs[:,:15].T)

150 (3, 150)
[[0.01 0.06 0.06]
 [0.01 0.06 0.14]
 [0.01 0.06 0.22]
 [0.01 0.06 0.3 ]
 [0.01 0.06 0.38]
 [0.01 0.14 0.06]
 [0.01 0.14 0.14]
 [0.01 0.14 0.22]
 [0.01 0.14 0.3 ]
 [0.01 0.14 0.38]
 [0.01 0.22 0.06]
 [0.01 0.22 0.14]
 [0.01 0.22 0.22]
 [0.01 0.22 0.3 ]
 [0.01 0.22 0.38]]


In [20]:
np.savetxt(os.path.join(outpath,'receiver.dat'),
           gridded_recs.T.astype(np.float32), 
           fmt='%.4f')